# 数据抓取说明报告

## 实验目标
本报告详细说明GitHub仓库数据抓取过程，包括使用的API接口、速率限制处理策略、数据获取流程以及技术实现细节。

## 1. 数据抓取概述

### 1.1 抓取目标
- **仓库**: yii2项目 (yiisoft/yii2)
- **数据类型**: Pull Request相关数据
- **数据规模**: 约8000+个PR记录
- **时间范围**: 项目历史所有PR数据

### 1.2 抓取数据表
| 数据类型 | 文件名 | 数据规模 | 数据内容 | 用途 |
|---------|--------|----------|----------|------|
| PR基础信息 | PR_info.xlsx | 1.2MB | PR基本信息、状态、时间 | 基础数据源 |
| PR详细信息 | PR_info_add_conversation.xlsx | 1.5MB | 包含对话信息的PR详情 | 时间分析 |
| PR特征数据 | PR_features.xlsx | 38MB | 代码统计、文件变更等特征 | 特征工程 |
| PR提交信息 | PR_commit_info.xlsx | 3.0MB | 提交记录、代码变更 | 代码分析 |
| PR评论信息 | PR_comment_info.xlsx | 2.5MB | 评论内容、时间戳 | 评审过程分析 |
| 作者特征 | author_features.xlsx | 880KB | 作者历史表现、活跃度 | 作者特征工程 |
| 评审者特征 | reviewer_features.xlsx | 988KB | 评审者信息、评审历史 | 评审者分析 |
| 项目特征 | project_features.xlsx | 2.4MB | 项目整体统计信息 | 项目状态分析 |
| Vue特征 | Vue_features.csv | 2.6MB | Vue相关PR特征 | 特定技术栈分析 |
| 时间字典 | pr_time_dict.txt | 1.2MB | PR时间映射关系 | 时间特征处理 |


## 2. GitHub API接口说明

### 2.1 使用的API端点


In [1]:
# GitHub API接口配置
API_ROOT = "https://api.github.com"

# 主要API端点
api_endpoints = {
    "PR列表": f"{API_ROOT}/repos/yiisoft/yii2/pulls",
    "PR详情": f"{API_ROOT}/repos/yiisoft/yii2/pulls/{{pr_number}}",
    "PR文件": f"{API_ROOT}/repos/yiisoft/yii2/pulls/{{pr_number}}/files",
    "PR评论": f"{API_ROOT}/repos/yiisoft/yii2/pulls/{{pr_number}}/reviews",
    "贡献者": f"{API_ROOT}/repos/yiisoft/yii2/contributors"
}

print("=== GitHub API 端点 ===")
for name, url in api_endpoints.items():
    print(f"{name}: {url}")

print(f"\nAPI基础URL: {API_ROOT}")
print("认证方式: Bearer Token (Personal Access Token)")


=== GitHub API 端点 ===
PR列表: https://api.github.com/repos/yiisoft/yii2/pulls
PR详情: https://api.github.com/repos/yiisoft/yii2/pulls/{pr_number}
PR文件: https://api.github.com/repos/yiisoft/yii2/pulls/{pr_number}/files
PR评论: https://api.github.com/repos/yiisoft/yii2/pulls/{pr_number}/reviews
贡献者: https://api.github.com/repos/yiisoft/yii2/contributors

API基础URL: https://api.github.com
认证方式: Bearer Token (Personal Access Token)


### 2.2 API请求参数

#### PR列表API参数
- `state`: PR状态 (all/open/closed)
- `sort`: 排序方式 (created/updated/popularity)
- `direction`: 排序方向 (asc/desc)
- `page`: 页码 (分页获取)
- `per_page`: 每页数量 (最大100)

#### 分页策略
- 使用`page`和`per_page`参数实现分页
- 每页最多100条记录
- 按创建时间倒序获取最新PR


In [2]:
# API请求示例
import requests
import time
from datetime import datetime

def demonstrate_api_usage():
    """演示API使用方式"""
    
    # 请求头配置
    headers = {
        "Accept": "application/vnd.github+json",
        "User-Agent": "ML-Research/1.0",
        "Authorization": "Bearer YOUR_TOKEN_HERE"  # 需要替换为实际token
    }
    
    # PR列表请求示例
    pr_list_url = "https://api.github.com/repos/yiisoft/yii2/pulls"
    params = {
        "state": "all",
        "sort": "created",
        "direction": "desc",
        "per_page": 10,  # 仅获取10条用于演示
        "page": 1
    }
    
    print("=== API请求示例 ===")
    print(f"URL: {pr_list_url}")
    print(f"参数: {params}")
    print(f"请求头: {headers}")
    
    # 注意：这里不实际发送请求，避免触发速率限制
    print("\n注意：实际使用时需要有效的GitHub Token")

demonstrate_api_usage()


=== API请求示例 ===
URL: https://api.github.com/repos/yiisoft/yii2/pulls
参数: {'state': 'all', 'sort': 'created', 'direction': 'desc', 'per_page': 10, 'page': 1}
请求头: {'Accept': 'application/vnd.github+json', 'User-Agent': 'ML-Research/1.0', 'Authorization': 'Bearer YOUR_TOKEN_HERE'}

注意：实际使用时需要有效的GitHub Token


## 3. 速率限制处理策略

### 3.1 GitHub API速率限制

GitHub API对未认证和已认证用户有不同的速率限制：

| 用户类型 | 每小时请求数 | 每分钟请求数 | 备注 |
|---------|-------------|-------------|------|
| 未认证 | 60 | 1 | 严重限制 |
| 已认证 | 5,000 | 83 | 需要Personal Access Token |
| 企业版 | 15,000 | 250 | 需要企业账户 |

### 3.2 速率限制检测与处理


In [4]:
# 速率限制处理策略实现
import time
import requests
from datetime import datetime, timedelta

class RateLimitHandler:
    """GitHub API速率限制处理器"""
    
    def __init__(self):
        self.rate_limit_remaining = 5000  # 假设已认证用户
        self.rate_limit_reset = None
        self.last_request_time = 0
        self.min_interval = 0.1  # 最小请求间隔（秒）
    
    def check_rate_limit(self, response):
        """检查响应头中的速率限制信息"""
        if 'X-RateLimit-Remaining' in response.headers:
            self.rate_limit_remaining = int(response.headers['X-RateLimit-Remaining'])
            print(f"剩余请求数: {self.rate_limit_remaining}")
        
        if 'X-RateLimit-Reset' in response.headers:
            reset_timestamp = int(response.headers['X-RateLimit-Reset'])
            self.rate_limit_reset = datetime.fromtimestamp(reset_timestamp)
            print(f"速率限制重置时间: {self.rate_limit_reset}")
    
    def handle_rate_limit_exceeded(self, response):
        """处理速率限制超出情况"""
        if response.status_code == 403 and 'X-RateLimit-Remaining' in response.headers:
            remaining = int(response.headers['X-RateLimit-Remaining'])
            if remaining == 0:
                reset_timestamp = int(response.headers['X-RateLimit-Reset'])
                sleep_seconds = max(0, reset_timestamp - int(time.time()) + 1)
                print(f"速率限制达到，等待 {sleep_seconds} 秒...")
                time.sleep(sleep_seconds)
                return True
        return False
    
    def enforce_min_interval(self):
        """强制最小请求间隔"""
        current_time = time.time()
        elapsed = current_time - self.last_request_time
        if elapsed < self.min_interval:
            sleep_time = self.min_interval - elapsed
            time.sleep(sleep_time)
        self.last_request_time = time.time()
    
    def make_request(self, session, url, params=None, max_retries=3):
        """发送请求并处理速率限制"""
        for attempt in range(max_retries):
            self.enforce_min_interval()
            
            try:
                response = session.get(url, params=params)
                self.check_rate_limit(response)
                
                if response.status_code == 200:
                    return response
                elif self.handle_rate_limit_exceeded(response):
                    continue  # 重试
                elif response.status_code >= 500:
                    print(f"服务器错误 {response.status_code}，重试 {attempt + 1}/{max_retries}")
                    time.sleep(2 ** attempt)  # 指数退避
                    continue
                else:
                    print(f"请求失败: {response.status_code}")
                    return response
                    
            except Exception as e:
                print(f"请求异常: {e}")
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)
                    continue
                raise
        
        return None

# 演示速率限制处理
print("=== 速率限制处理策略 ===")
handler = RateLimitHandler()
print(f"初始剩余请求数: {handler.rate_limit_remaining}")
print(f"最小请求间隔: {handler.min_interval} 秒")
print("处理策略: 指数退避 + 速率限制检测 + 最小间隔控制")


=== 速率限制处理策略 ===
初始剩余请求数: 5000
最小请求间隔: 0.1 秒
处理策略: 指数退避 + 速率限制检测 + 最小间隔控制
